# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [3]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json
import time
# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [4]:
city_data = pd.read_csv("../output_data/clean_city_data.csv")
city_data

,Unnamed: 0,City,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
0,0,Busselton,-33.65,115.33,55.92,73,7,13.91
1,1,Ascension,30.22,-90.90,78.19,83,34,4.03
2,2,Huarmey,-10.07,-78.15,59.61,85,10,9.08
3,3,Bitung,1.45,125.18,87.80,66,20,2.24
4,4,Faanui,-16.48,-151.75,77.05,83,99,17.78
...,...,...,...,...,...,...,...,...
548,548,Chara,56.91,118.26,56.88,80,100,2.73
549,549,Pangody,65.85,74.49,53.78,85,100,10.71
550,550,Marsh Harbour,26.54,-77.06,82.78,80,100,17.54
551,551,Zabid,14.20,43.32,90.75,36,95,3.11


### Humidity Heatmap
* Configure gmaps.

In [5]:
gmaps.configure(api_key=g_key)

* Use the Lat and Lng as locations and Humidity as the weight.

In [6]:
locations = city_data[["Latitude", "Longitude"]]
weight = city_data["Humidity"]

In [24]:
# Customize the size of the figure
figure_layout = {
    'width': '600px',
    'height': '450px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout, center=(46.0, -5.0), zoom_level=2)

* Add Heatmap layer to map.

In [25]:
heat_layer = gmaps.heatmap_layer(locations, weight, dissipating=False, max_intensity=np.max(weight), point_radius=3)

In [26]:
# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='450px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
hotel_df = city_data.loc[(city_data["Wind Speed"] <= 15) & (city_data["Cloudiness"] <= 10) &(city_data["Temp"] >= 75) & (city_data["Temp"] <= 95) & (city_data["Humidity"] <= 70)].dropna()

hotel_df

,Unnamed: 0,City,Latitude,Longitude,Temp,Humidity,Cloudiness,Wind Speed
8,8,Jalu,29.03,21.55,77.31,54,0,6.82
39,39,Ürümqi,43.80,87.60,91.40,21,0,6.71
60,60,Laguna,38.42,-121.42,84.06,34,1,13.87
107,107,Pasni,25.26,63.47,89.38,66,0,10.00
113,113,Dahuk,36.87,42.99,78.08,20,0,2.98
138,138,Abū Zabad,12.35,29.25,75.72,68,8,14.83
145,145,Pierre,44.37,-100.35,91.33,40,1,3.36
208,208,Riyadh,24.69,46.72,87.80,15,0,6.93
213,213,Alyangula,-13.85,136.42,82.40,48,7,5.82
216,216,Kumul,42.80,93.45,86.36,17,0,7.76


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
